# Analyse Données Géographiques

- chargement des données
    - requête OSM
    - aire wikidata
    - noeuds, chemins, relations
- DataFrame dès le début ?
    - conversion JSON à un tableau dataframe
- Données intéressantes :
    - nombre d'éléments
    - nombre de tags total
    - nombre de tags par élément + classement
    - (tag overlap...)
- données analyse à part
    - restaurants
    - magasins
    - montagnes : et encore parce que peut-être colinne intéressante ?
    - arbre qui n'ont que un tag ?

# Contenu d'un élément
Un élément a un __identifiant overpass__, une __latitude__ et __longitude__, un __type__ (_noeud, chemin, relations_), un __tag__ et une __valeur__.

__Questions clés :__ quels sont mes concepts abstraits, comment les construire de manière la plus globale possible, quelles sont les fonctionnalités et les héritages ?

éléments intéressants : affichage sur une carte, affichage des numéros, détection d'accumulation de noeuds par coordonnées géographiques... (typiquement d'un même tag)

__Analyse :__
Si on crée un paysage. Cela vaudrait-il la peine d'avoir un seul maître qui pourrait intéragir avec d'autres objets pour envoyer des messages par exemple ou est-ce qu'il faut un peu séparer les tâches ?

Les étapes principales sont le chargement avec overpass (OL et REQUEST). Puis les étapes d'analyse de fréquence et divers filtrages. Et encore découverte. Aussi, si on a un résultat on voudrait pouvoir le grapher sur dune carte.

Option : plusieurs REQUESTS et ajouter une colonne par exemple...

__Note :__ les données numériques seront les coordonnées (détection d'accumulation) et les différentes élévations (statistique)

Il faudra toujours avoir le contrôle sur combient d'éléments sont disponibles et combien il en reste si on applique un filtre.

# Réflextion future
Est-ce que je peux faire un pont sur les requête sur wikidata ? : idée jouer avec héritage ?

In [51]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class Loader:
    '''JSONel : en JSON les éléments'''
    def __init__(self,area='Q78'):
        # option : rajouter des aires prédéfinies avec des clés
        # option 2 : étendre requêtes aux chemins et relations
        r_url = '%5Bout%3Ajson%5D%3B%0D%0Aarea%5Bwikidata%3D'+area+'%5D%3B%0D%0Anode%28area%29%5B%7E%27%5Ba-z0-9%5D%27%7E%27%5Ba-z0-9%5D%27%5D%3B%0D%0Aout%3B'
        r = requests.get('https://www.overpass-api.de/api/interpreter?data='+r_url)
        jsonobject = r.json()
        Loader.JSONel = jsonobject['elements']
    def get_json(self):
        return self.JSONel
    def get_DataFrame(self):
        table = list()
        for element in self.JSONel: # elements isolé
            for tag, value in element['tags'].items():
                table.append([element['id'],element['lat'],element['lon'],element['type'],tag,value])
        return pd.DataFrame(table,columns=['id','lat','lon','type','tag','value'])

def filterOV(df,drop=True,column1='tag',value1='',column2='',value2=''):
    # option 1 : enlever tous éléments liés à une colonne qui a une valeur
    if(drop):
        if(value2==''):
            identifiers = df[df[column1]==value1]['id']
        else:
            b_val1 = np.array(df[column1]==value1)
            b_val2 = np.array(df[column2]==value2)
            # liste des identifiants interdits
            mask = pd.Series(b_val1&b_val2)
            mask.index = df.index
            identifiers = df[mask]['id'].unique()
        id_tot = df['id']
        id_diff = pd.Series(np.setdiff1d(np.array(id_tot),np.array(identifiers)))
        return df[df['id'].isin(id_diff)]
    # option 2 : ne laisser que les éléments liés au couple colonne et valeur
    else:
        if(value2==''):
            identifiers = df[df[column1]==value1]['id']
        else:
            b_val1 = np.array(df[column1]==value1)
            b_val2 = np.array(df[column2]==value2)
            mask = pd.Series(b_val1&b_val2)
            identifiers = df[mask]['id'].unique()
        return df[df['id'].isin(identifiers)]
            
# option : si on crée une forme spécifique à OL on peut instancier héritier ?
class Master:
    # initialisation
    def __init__(self,area='Q78'): # Q78 == Basel
        Master.io = list()
        Master.df = Loader().get_DataFrame()
        Master.dfc = Master.df.copy()
        self.get_statsDF()
        Master.originaldf = Master.df.copy()
        
    #tags
    # sauver un tag intéressant
    def save_tag(self,tag):
        if tag in self.io:
            print('note : déjà enregistré')
        else:
            # option : vérifier type de tag
            if(type(tag)=='str'):
                self.io.append(tag)
            elif(type(tag)=='list'):
                for item in tag:
                    if item not in self.io:
                        self.io.append(item)
    # obtenir les tags intéressants
    def get_tags(self):
        return self.io
    
    # reset DataFrame
    def reset(self):
        self.df = self.originaldf.copy()
    
    # statistiques
    # statistiques par rapport au dernier updtate : longueur du set, nb de noeuds, nb de tags
    def get_statsDF(self):
        print("Len df : {}. Difference : {}".format(len(self.dfc),len(self.df)-len(self.dfc)))
        print("Nb items : {}. Difference : {}".format(len(self.dfc['id'].unique()),len(self.df['id'].unique())-len(self.dfc['id'].unique())))
        print("Nb tags : {}. Difference : {}".format(len(self.dfc['tag'].unique()),len(self.df['tag'].unique())-len(self.dfc['tag'].unique())))
        self.df = self.dfc.copy()
    # statistiques des valeurs associés
    def get_counts_ass(self, column='tag', value='amenity', columnToAnalyse='value'):
        return self.dfc[self.dfc[column]==value][columnToAnalyse].value_counts()
    # statistique d'une colonne
    def get_counts_column(self, column='tag'):
        return self.dfc[column].value_counts()
    
    #filtre par rapport à une colonne ou deux
    def filterdf(self,drop=True,column1='tag',value1='',column2='value',value2='',replace=True):
        result = filterOV(self.dfc,drop=drop,column1=column1,value1=value1,column2=column2,value2=value2)
        if(replace):
            self.dfc = result
        if(replace==False):
            return result
        else:
            self.get_statsDF()
            return ''
    #filtre sans remplacement par rapport id
    def filterId(self, id):
        return self.dfc[self.dfc['id']==id]
    
        
        

In [2]:
# instancier
m1 = Master()

# filtrer les restaurants
m1.filterdf(value1='amenity',column2='value',value2='restaurant')

Len df : 124079. Difference : 0
Nb items : 30565. Difference : 0
Nb tags : 432. Difference : 0
Len df : 122490. Difference : 1589
Nb items : 30292. Difference : 273
Nb tags : 429. Difference : 3


''

In [3]:
# m1.get_counts_ass(column='tag',value='amenity',columnToAnalyse='value')
# m1.filterdf(value1='bench',column1='value',replace=False)
# m1.filterId(5820379660)
# m1.get_counts_column(column='tag')

In [4]:
#m1.get_counts_ass(column='tag',value='shop',columnToAnalyse='value')[m1.get_counts_ass(column='tag',value='shop',columnToAnalyse='value')>8]
#m1.filterdf(value1='leaf_type',column1='tag',replace=False, drop=False)['tag'].value_counts()

m1.filterdf(value1='leaf_type',column1='tag',replace=True, drop=True)

Len df : 121671. Difference : 819
Nb items : 29935. Difference : 357
Nb tags : 426. Difference : 3


''

In [5]:
m1.save_tag('second_hand (value, not a tag)')
m1.save_tag('shop=books')
m1.save_tag('amenity=pharmacy')
m1.save_tag('amenity=waste_basket')
m1.save_tag('amenity=toilets')
m1.save_tag('amenity=ferry_terminal')
m1.save_tag('amenity=fuel')
m1.save_tag('amenity=library')
m1.save_tag('amenity=fountain')


In [11]:
m1.filterdf(value1='amenity',column1='tag',value2='vending_machine',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='post_box',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='waste_basket',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='parkingpharmacy',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='pharmacy',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='fast_food',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='bank',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='car_sharing',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='fuel',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='ferry_terminal',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='car_sharing',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='school',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='car_sharing',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='nightclub',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='car_sharing',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='doctors',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='hospital',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='parking_entrance',column2='value',replace=True, drop=True)

Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. Difference : 0
Nb tags : 404. Difference : 0
Len df : 119586. Difference : 0
Nb items : 29392. 

''

In [14]:
m1.get_counts_ass(column='tag',value='amenity',columnToAnalyse='value')
m1.filterdf(value1='amenity',column1='tag',value2='parking',column2='value',replace=True, drop=True)
m1.filterdf(value1='amenity',column1='tag',value2='post_office',column2='value',replace=True, drop=True)
print()

Len df : 119361. Difference : 0
Nb items : 29335. Difference : 0
Nb tags : 401. Difference : 0
Len df : 119361. Difference : 0
Nb items : 29335. Difference : 0
Nb tags : 401. Difference : 0



In [35]:
m1.save_tag('archaeological_site, monument, memorial, monastery')
m1.filterdf(value1='historic',column1='tag',column2='value',replace=False, drop=False)[m1.filterdf(value1='historic',column1='tag',column2='value',replace=False, drop=False)['tag']=='historic']['value'].value_counts()

Len df : 119361. Difference : 0
Nb items : 29335. Difference : 0
Nb tags : 401. Difference : 0
Len df : 119361. Difference : 0
Nb items : 29335. Difference : 0
Nb tags : 401. Difference : 0


archaeological_site    13
monument                3
memorial                3
monastery               1
boundary_stone          1
Name: value, dtype: int64

# Idée d'analyse
- analyser séparément - par exemple par couleur
    - restaurants (peut-être truc cool)
    - magasin (spéciaux, second-hand)
    - lieux sympas
        - fontaines
        - sculptures
        - historique
        - bibliothèques
        - musée
        - bâtiments religieux
- éléments naturels
    - montagnes = nom, hauteur

In [34]:
#m1.get_counts_ass(column='tag',value='tourism',columnToAnalyse='value')

In [37]:
m1.save_tag('tourism=museum')
m1.get_counts_ass(column='value',value='museum',columnToAnalyse='tag')

tourism    18
Name: tag, dtype: int64

In [40]:
m1.get_counts_ass(column='tag',value='leisure',columnToAnalyse='value')

playground         44
fitness_station    17
pitch               8
picnic_table        7
sports_centre       6
fitness_centre      4
marina              2
firepit             1
slipway             1
park                1
swimming_pool       1
hackerspace         1
dance               1
Name: value, dtype: int64

In [41]:
m1.save_tag('leisure')

# Conclusion
Trouver une façon + propre de présenter les données. Par exemple avec um prompter et une marche à suivre.
Continuer :
- intégrer l'analyse avec les chemins et les relations
- intégrer l'analyse avec wikidata

In [46]:
m1.get_counts_ass(column='tag',value='religion',columnToAnalyse='tag')

religion    14
Name: tag, dtype: int64

In [50]:
m1.filterdf(value1='religion',column1='tag',replace=False, drop=False)['value'].value_counts()

Len df : 119361. Difference : 0
Nb items : 29335. Difference : 0
Nb tags : 401. Difference : 0


place_of_worship                                                                                                                                                                                                                                  14
christian                                                                                                                                                                                                                                         12
Basel                                                                                                                                                                                                                                              5
roman_catholic                                                                                                                                                                                                                                     2
Evangelisch-Lutheris

In [55]:
m1.filterdf(value1='artist_name',column1='tag',replace=False, drop=False)['tag'].value_counts()

Len df : 119361. Difference : 0
Nb items : 29335. Difference : 0
Nb tags : 401. Difference : 0


artwork_type    8
name            8
artist_name     8
tourism         8
start_date      7
material        7
name:en         1
Name: tag, dtype: int64

In [54]:
m1.save_tag('artist_name')
m1.save_tag('artwork_type')

In [59]:
m1.filterdf(value1='museum',column1='value',replace=False, drop=False)['tag'].value_counts()

Len df : 119361. Difference : 0
Nb items : 29335. Difference : 0
Nb tags : 401. Difference : 0


name                    18
tourism                 18
wheelchair              14
addr:postcode           13
website                 13
addr:city               13
addr:street             13
addr:housenumber        13
addr:country             9
wikipedia                3
wikidata                 3
opening_hours            2
payment:cash             1
name:de                  1
internet_access          1
operator                 1
payment:debit_cards      1
payment:credit_cards     1
building                 1
contact:website          1
internet_access:fee      1
email                    1
phone                    1
start_date               1
Name: tag, dtype: int64